In [ ]:
!pip3 install transformers optimum-intel evaluate neural-compressor -q
!pip install onnx
!pip install faiss-cpu
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Allowed to make changes.
import collections
import json
import pandas as pd
import re
import string
import timeit
from ast import literal_eval
import math
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from optimum.intel.neural_compressor import INCModelForQuestionAnswering

In [ ]:
# Allowed to make changes.

# Pre-processing cell. You can use this cell to pre-process input data or load
# your models.
import torch
import faiss
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-cos-v1')
# Dummy code.



question_loc=r"/content/drive/MyDrive/sample_eval_test/5_theme/sample_input_question_30.csv"

theme_loc=r"/content/drive/MyDrive/sample_eval_test/5_theme/sample_theme_interval_30.csv"

ground_loc=r"/content/drive/MyDrive/sample_eval_test/5_theme/sample_ground_truth_values_30.csv"

df_p=pd.read_csv("/content/drive/MyDrive/sample_eval_test/5_theme/sample_input_paragraph_30.csv")

# print(df_p.head(-5))

theme_dict={}
t=0
for i in df_p["theme"].unique() :
    start=t
    # i=j.lower()
    print()
    print("theme "+i)
    if(t==len(df_p)):
            break
    while(df_p["theme"][t]==i): # use lower here if you  wish to change
        t=t+1
        if(t==len(df_p)):
            break
    theme_dict[i]={}
    theme_dict[i]["df"]=df_p.iloc[start:t,:]
    theme_dict[i]["df"]=theme_dict[i]["df"].reset_index(drop=True)
    # key obesrvation : some theme have very less paragraph (try linear search)
    text_vector=theme_dict[i]["df"]["paragraph"].to_list()
    embedding_vector=embedding_model.encode(text_vector)
    vector_length=embedding_vector.shape[1]
    no_of_vector=embedding_vector.shape[0]

    if(no_of_vector<20):
      no_of_cell=1
    elif(no_of_vector<100):
      no_of_cell=10
    else :
      no_of_cell=20

    theme_dict[i]["quantizer_index"]=faiss.IndexFlatL2(vector_length)
    theme_dict[i]["index"] = faiss.IndexIVFFlat(theme_dict[i]["quantizer_index"],vector_length,no_of_cell)   # build the index
    # print(index.is_trained)
    theme_dict[i]["index"].train(embedding_vector)
    theme_dict[i]["index"].add(embedding_vector)
    theme_dict[i]["index"].nprobe=2
    print("no of paragraph in theme "+str(theme_dict[i]["index"].ntotal))
    print()





paragraph_ds = []




global_model_text = r"C:\Users\RAJESH SOLANKI\Desktop\dev rev inter iit\retrival task\spanbert_quant"


# Here, you can load the existing QA pairs for themes and pre-process it.
# Sample QA pairs: https://drive.google.com/file/d/1HORTMN3UrPyfcibMZIA6tVnkgQXlZfWn/view?usp=share_link
questions = []



theme Spectre_(2015_film)
no of paragraph in theme 43


theme To_Kill_a_Mockingbird
no of paragraph in theme 60


theme Kanye_West
no of paragraph in theme 78


theme Dog
no of paragraph in theme 74


theme Separation_of_powers_under_the_United_States_Constitution
no of paragraph in theme 29


theme Boston
no of paragraph in theme 79


theme University_of_Kansas
no of paragraph in theme 36


theme Classical_music
no of paragraph in theme 68


theme List_of_numbered_streets_in_Manhattan
no of paragraph in theme 50


theme Near_East
no of paragraph in theme 59


theme Wood
no of paragraph in theme 69


theme Mexico_City
no of paragraph in theme 87


theme Data_compression
no of paragraph in theme 26


theme Somerset
no of paragraph in theme 37


theme Edmund_Burke
no of paragraph in theme 40


theme Samoa
no of paragraph in theme 25


theme Nigeria
no of paragraph in theme 55


theme Sichuan
no of paragraph in theme 26


theme War_on_Terror
no of paragraph in theme 31


theme Labour_Par

In [ ]:
# Allowed to make changes.
from transformers import pipeline




def get_theme_model(theme):
  global_model=pipeline("question-answering",model="mrm8488/spanbert-finetuned-squadv2",tokenizer="mrm8488/spanbert-finetuned-squadv2",device=0)
  return global_model



In [ ]:
theme_dict_save = '/content/drive/MyDrive/sample_eval_test/dev_rev_test_data/test_data/theme_dict2.obj'

In [ ]:
# save
import pickle
dbfile = open(theme_dict_save, 'wb')
pickle.dump(theme_dict, dbfile)
dbfile.close()

In [ ]:
# Allowed to make changes.
def pred_theme_ans(questions, theme_model, pred_out):
  theme = questions[0]["theme"]
  paragraph_dict=theme_dict[theme]
  no_of_neighbors=1 # for sample dataset
  if(no_of_neighbors>len(paragraph_dict["df"]["paragraph"])):
    no_of_neighbors=len(paragraph_dict["df"]["paragraph"])
    print("no of neighbors changed (not enough paragraph) to : "+str(no_of_neighbors))

  for question in questions:

    question_embedding=embedding_model.encode([question["question"]])

    D, Index_of_paragraph = paragraph_dict["index"].search(question_embedding, no_of_neighbors)

    p_id=-1
    ans_output=""
    ans_confidence=0
    for i in Index_of_paragraph[0] :

        if(i<0):
          continue
        text=paragraph_dict["df"]["paragraph"][i]
        #print(text)
        ans_dictonary= theme_model({
        'context': text,
        'question': question["question"]
        })

        # if(ans_dictonary["score"]>=0.1):
        if(ans_dictonary["score"]>ans_confidence):
          ans_confidence=ans_dictonary["score"]
          ans_output=ans_dictonary["answer"]

          p_id=paragraph_dict["df"]["id"][i]
    #-------------------------------------
    # add your prediction methodology here.
    #-------------------------------------
    # Dummy method.
    ans = {}
    ans["question_id"] = question["id"]
    ans["paragraph_id"] = p_id
    ans["answers"]=ans_output

    pred_out.append(ans)
  print("done")

In [ ]:
# NOT allowed to make changes.


# All theme prediction.

questions = json.loads(pd.read_csv(question_loc).to_json(orient="records"))
theme_intervals = json.loads(pd.read_csv(theme_loc).to_json(orient="records"))
pred_out = []
theme_inf_time = {}
theme_array_inf=[]
for theme_interval in theme_intervals:

  theme_ques = questions[int(theme_interval["start"]) - 1: int(theme_interval["end"])]
  theme = theme_ques[0]["theme"]
  # theme=theme.lower()
  print(theme)
  theme_array_inf.append(theme)
  # Load model fine-tuned for this theme.
  theme_model = get_theme_model(theme)
  #pred_theme_ans(theme_ques, theme_model, pred_out)
  execution_time = timeit.timeit(lambda: pred_theme_ans(theme_ques, theme_model, pred_out), number=1)
  theme_inf_time[theme_interval["theme"]] = execution_time * 1000 # in milliseconds.
  print("inference time : "+str(theme_inf_time[theme_interval["theme"]]))
pred_df = pd.DataFrame.from_records(pred_out)
pred_df.fillna(value='', inplace=True)
# Write prediction to a CSV file. Teams are required to submit this csv file.
pred_df.to_csv('sample_output_prediction.csv', index=False)
print(theme_inf_time)

Spectre_(2015_film)
done
inference time : 9486.81619600029
To_Kill_a_Mockingbird
done
inference time : 4264.442852000229
Kanye_West
done
inference time : 9106.311464999635
Dog
done
inference time : 6418.949612999768
Separation_of_powers_under_the_United_States_Constitution
done
inference time : 5159.755534999931
Boston
done
inference time : 6989.805478999642
University_of_Kansas
done
inference time : 7369.9062360001335
Classical_music
done
inference time : 5166.059706999931
List_of_numbered_streets_in_Manhattan
done
inference time : 4060.4669170002126
Near_East
done
inference time : 4305.223495000064
Wood
done
inference time : 5203.550633999839
Mexico_City
done
inference time : 7925.897237000299
Data_compression
done
inference time : 3475.819836000028
Somerset
done
inference time : 7107.611318000181
Edmund_Burke
done
inference time : 7042.520440000317
Samoa
done
inference time : 4963.095737000003
Nigeria
done
inference time : 4232.588492000104
Sichuan
done
inference time : 7475.5843590

In [ ]:
print(theme_inf_time)
pred_df.to_csv('sample_output_prediction.csv', index=False)

{'Spectre_(2015_film)': 9486.81619600029, 'To_Kill_a_Mockingbird': 4264.442852000229, 'Kanye_West': 9106.311464999635, 'Dog': 6418.949612999768, 'Separation_of_powers_under_the_United_States_Constitution': 5159.755534999931, 'Boston': 6989.805478999642, 'University_of_Kansas': 7369.9062360001335, 'Classical_music': 5166.059706999931, 'List_of_numbered_streets_in_Manhattan': 4060.4669170002126, 'Near_East': 4305.223495000064, 'Wood': 5203.550633999839, 'Mexico_City': 7925.897237000299, 'Data_compression': 3475.819836000028, 'Somerset': 7107.611318000181, 'Edmund_Burke': 7042.520440000317, 'Samoa': 4963.095737000003, 'Nigeria': 4232.588492000104, 'Sichuan': 7475.584359000095, 'War_on_Terror': 6372.976611000013, 'Labour_Party_(UK)': 6032.32302799961, 'IBM': 4526.839146999919, 'Friedrich_Hayek': 4260.655437999958, 'Idealism': 5666.692237999996, 'Cyprus': 4217.244389999905, 'Tucson,_Arizona': 7454.048687000068, 'Premier_League': 10668.166381999981, 'Association_football': 4810.342911000134,

In [ ]:
# pred_df.to_csv('/content/drive/MyDrive/sample_eval_test/dev_rev_test_data/test_data/update2/output_prediction.csv', index=False)


In [ ]:
# NOT allowed to make changes.

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def calc_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def calc_max_f1(predicted, ground_truths):
  max_f1 = 0
  if len(ground_truths) == 0:
    return len(predicted) == 0
  for ground_truth in ground_truths:
    f1 = calc_f1(predicted, ground_truth)
    max_f1 = max(max_f1, f1)
  return max_f1

In [ ]:
# NOT allowed to make changes.

# Evaluation methodology.
metrics = {}
pred = pd.read_csv(r"sample_output_prediction.csv")
pred.fillna(value='', inplace=True)
truth = pd.read_csv(ground_loc)
truth.fillna(value='', inplace=True)
truth.paragraph_id = truth.paragraph_id.apply(literal_eval)
truth.answers = truth.answers.apply(literal_eval)
questions = pd.read_csv(question_loc)
for idx in pred.index:
  q_id = pred["question_id"][idx]
  q_rows = questions.loc[questions['id'] == q_id].iloc[-1]
  theme = q_rows["theme"]
  predicted_paragraph = pred["paragraph_id"][idx]
  predicted_ans = pred["answers"][idx]

  if theme not in metrics.keys():
    metrics[theme] = {"true_positive": 0, "true_negative": 0, "total_predictions": 0, "f1_sum": 0}

  truth_row = truth.loc[truth['question_id'] == q_id].iloc[-1]
  truth_paragraph_id = [ int(i) for i in truth_row["paragraph_id"] ]
  # print()
  # print("predicted_paragraph : "+str(predicted_paragraph))
  # print(truth_paragraph_id)
  # print()
  if predicted_paragraph in truth_paragraph_id:
    # Increase TP for that theme.
    metrics[theme]["true_positive"] = metrics[theme]["true_positive"] + 1
  # -1 prediction in case there is no paragraph which can answer the query.
  if predicted_paragraph == -1 and truth_row["paragraph_id"] == []:
    # Increase TN.
    metrics[theme]["true_negative"] = metrics[theme]["true_negative"] + 1
  # Increase total predictions for that theme.
  metrics[theme]["total_predictions"] = metrics[theme]["total_predictions"] + 1
  f1 = calc_max_f1(predicted_ans, truth_row["answers"])
  metrics[theme]["f1_sum"] = metrics[theme]["f1_sum"] + f1

In [ ]:
# NOT allowed to make changes.

# Final score.
inf_time_threshold = 1000.0 # milliseconds.
final_para_score = 0.0
final_qa_score = 0.0
# Weight would stay hidden from teams.
#theme_weights = {"Kubernetes": 0.5, "ChatGPT": 0.4, "Football world cup": 0.1}
# theme_weights = {"Uranium": 0.2, "Bill_%26_Melinda_Gates_Foundation": 0.2, "Somerset": 0.2, "Rule_of_law": 0.2, "Bird": 0.2}
theme_weights = {}

for i in theme_array_inf:
  theme_weights[i]=1/len(theme_array_inf)

print(theme_weights)

for theme in metrics:
  inf_time_score = 1.0
  metric = metrics[theme]
  para_score = (metric["true_positive"] + metric["true_negative"]) / metric["total_predictions"]
  qa_score = metric["f1_sum"] / metric["total_predictions"]
  avg_inf_time = theme_inf_time[theme] / metric["total_predictions"]
  print()
  print("total inference time for that theme : "+str(theme_inf_time[theme])+"   total no of question   "+str(metric["total_predictions"]))
  print("avg inference time : "+str(avg_inf_time))
  print("qa score without accounting time for that theme :"+str(qa_score))
  if avg_inf_time > inf_time_threshold:

    inf_time_score = inf_time_threshold / avg_inf_time
  final_qa_score += theme_weights[theme] * inf_time_score * qa_score
  final_para_score += theme_weights[theme] * inf_time_score * para_score
print ("final paragraph pridiction score :"+str(final_para_score))
print ("final qa score :"+str(final_qa_score))


{'Spectre_(2015_film)': 0.03333333333333333, 'To_Kill_a_Mockingbird': 0.03333333333333333, 'Kanye_West': 0.03333333333333333, 'Dog': 0.03333333333333333, 'Separation_of_powers_under_the_United_States_Constitution': 0.03333333333333333, 'Boston': 0.03333333333333333, 'University_of_Kansas': 0.03333333333333333, 'Classical_music': 0.03333333333333333, 'List_of_numbered_streets_in_Manhattan': 0.03333333333333333, 'Near_East': 0.03333333333333333, 'Wood': 0.03333333333333333, 'Mexico_City': 0.03333333333333333, 'Data_compression': 0.03333333333333333, 'Somerset': 0.03333333333333333, 'Edmund_Burke': 0.03333333333333333, 'Samoa': 0.03333333333333333, 'Nigeria': 0.03333333333333333, 'Sichuan': 0.03333333333333333, 'War_on_Terror': 0.03333333333333333, 'Labour_Party_(UK)': 0.03333333333333333, 'IBM': 0.03333333333333333, 'Friedrich_Hayek': 0.03333333333333333, 'Idealism': 0.03333333333333333, 'Cyprus': 0.03333333333333333, 'Tucson,_Arizona': 0.03333333333333333, 'Premier_League': 0.0333333333